In [3]:
import math
import heapq 
import collections
from typing import *
import bisect

# 宝石补给

In [3]:
class Solution:
    def giveGem(self, gen: List[int], operations: List[List[int]]) -> int:
        for i, j in operations:
            num = gen[i]//2
            gen[i] -= num
            gen[j] += num
        return max(gen) - min(gen)

# 烹饪料理

In [12]:
class Solution:
    def perfectMenu(self, materials: List[int], cookbooks: List[List[int]], attribute: List[List[int]], limit: int) -> int:
        def xunzhao(materials, num, n, visited):
            ret = num if n >= limit else -1
            for i in range(len(cookbooks)):
                cancook = True
                for j, m in enumerate(cookbooks[i]):
                    materials[j] -= m
                    if materials[j] < 0:
                        cancook = False
                if cancook and i not in visited:
                    visited.add(i)
                    ret = max(ret, xunzhao(materials, num+attribute[i][0], n+attribute[i][1], visited))
                for j, m in enumerate(cookbooks[i]):
                    materials[j] += m
            return ret
        
        return xunzhao(materials, 0, 0, set())

In [13]:
meterials = [10,10,10,10,10]
cookbooks = [[1,1,1,1,1],[3,3,3,3,3],[10,10,10,10,10]]
attribute = [[5,5],[6,6],[10,10]]
limit = 1

In [14]:
a = Solution().perfectMenu(meterials, cookbooks, attribute, limit)
a

11

# 二叉搜索树染色

## I use Interval merging algorithm to deal with this problem

In [2]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
        
class Solution:
    def getNumber(self, root: Optional[TreeNode], ops: List[List[int]]) -> int:
        def xunzhao(root, i, j):
            if not root:
                return 0
            ret = 0
            if i<=root.val<=j:
                ret += 1
                ret += xunzhao(root.left, i, j)
                ret += xunzhao(root.right, i, j)
            elif root.val > j:
                ret += xunzhao(root.left, i, j)      
            else:
                ret += xunzhao(root.right, i, j)
            return ret
            
        ret = []
        re = 0
        for op in ops[::-1]:
            if ret == []:
                ret.append([op[1], op[2]])
                if op[0] == 1:
                    re += xunzhao(root, op[1], op[2])
            else:
                nn = []
                n = [op[1], op[2]]
                for i, r in enumerate(ret):
                    if r[0]<=n[0]<=r[1]:
                        if r[1] >= n[1]:
                            n = []
                            break
                        n = [r[1]+1, n[1]]
                    elif r[0] <= n[1] <= r[1]:
                        n = [n[0], r[0]-1]
                        break
                    elif n[0]<=r[0]<=n[1]:
                        nn.append([n[0], r[0]-1])
                        # ret[i][0] = n[0]
                        n = [r[1]+1, n[1]]
                if n:
                    nn.append(n)
                ret = ret + nn
                ret.sort()
                if op[0] == 1:
                    for nnn in nn:
                        re += xunzhao(root, nnn[0], nnn[1])
        return re

In [32]:
class RangeModule(object):
    def __init__(self):
        self.ranges = []

    # It also can use bisect
    def _bounds(self, left, right):
        i, j = 0, len(self.ranges) - 1
        for d in (100, 10, 1):
            while i + d - 1 < len(self.ranges) and self.ranges[i+d-1][1] < left:
                i += d
            while j >= d - 1 and self.ranges[j-d+1][0] > right:
                j -= d
        return i, j

    def addRange(self, left, right):
        i, j = self._bounds(left, right)
        if i <= j:
            left = min(left, self.ranges[i][0])
            right = max(right, self.ranges[j][1])
        self.ranges[i:j+1] = [(left, right)]

    def queryRange(self, left, right):
        i = bisect.bisect_left(self.ranges, (left, float('inf')))
        if i: i -= 1
        return (bool(self.ranges) and
                self.ranges[i][0] <= left and
                right <= self.ranges[i][1])

    def removeRange(self, left, right):
        i, j = self._bounds(left, right)
        merge = []
        for k in range(i, j+1):
            if self.ranges[k][0] < left:
                merge.append((self.ranges[k][0], left))
            if right < self.ranges[k][1]:
                merge.append((right, self.ranges[k][1]))
        self.ranges[i:j+1] = merge

In [37]:
# 1 <= left < right <= 1e9

from typing import Optional

# 动态开点
class Node:
    __slots__ = ('isTracked', 'lazy', 'left', 'right')

    def __init__(
        self,
        isTracked=False,  
        lazy=False,
        left: Optional['Node'] = None,
        right: Optional['Node'] = None,
    ) -> None:
        self.isTracked = isTracked
        self.lazy = lazy
        self.left = left
        self.right = right


class SegmentTree:
    def __init__(self) -> None:
        self._root = Node()

    def update(self, left: int, right: int, delta: bool) -> None:
        self._update(left, right, 0, int(1e9 + 10), self._root, delta)

    def query(self, left: int, right: int) -> bool:
        return self._query(left, right, 0, int(1e9 + 10), self._root)

    def _update(self, L: int, R: int, l: int, r: int, root: Node, delta: bool) -> None:
        if L <= l <= r <= R:
            root.isTracked = delta
            root.lazy = True
            return

        self._pushDown(root)
        mid = (l + r) >> 1
        if L <= mid:
            self._update(L, R, l, mid, root.left, delta)
        if R >= mid + 1:
            self._update(L, R, mid + 1, r, root.right, delta)
        self._pushUp(root)

    def _query(self, L: int, R: int, l: int, r: int, root: Node) -> bool:
        if L <= l <= r <= R:
            return root.isTracked

        self._pushDown(root)
        mid = (l + r) >> 1
        res = True
        if L <= mid:
            res = res and self._query(L, R, l, mid, root.left)
        if R >= mid + 1:
            res = res and self._query(L, R, mid + 1, r, root.right)
        return res

    def _pushUp(self, root: Node) -> None:
        root.isTracked =  (
            root.left and root.left.isTracked and root.right and root.right.isTracked
        )

    def _pushDown(self, root: Node) -> None:
        if not root.left:
            root.left = Node()
        if not root.right:
            root.right = Node()
        if root.lazy:
            root.left.lazy = root.right.lazy = True
            root.left.isTracked = root.right.isTracked = root.isTracked
            root.lazy = False


class RangeModule:
    def __init__(self):
        self.tree = SegmentTree()

    def addRange(self, left: int, right: int) -> None:
        """添加 半开区间 [left, right)"""
        self.tree.update(left, right - 1, True)

    def queryRange(self, left: int, right: int) -> bool:
        """ 只有在当前正在跟踪区间 [left, right) 中的每一个实数时，才返回 true"""
        return self.tree.query(left, right - 1)

    def removeRange(self, left: int, right: int) -> None:
        """ 停止跟踪 半开区间 [left, right)"""
        self.tree.update(left, right - 1, False)

In [33]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None
class Solution:
    def getNumber(self, root: Optional[TreeNode], ops: List[List[int]]) -> int:
        def dfs(root, R):
            if not root:
                return
            dfs(root.left)
            R.queryRange(root.val, root.val+1)
            dfs(root.right)
            
        R = RangeModule()
        for i, l, r in ops:
            if i == 1:
                R.addRange(l, r+1)
            else:
                R.removeRange(l, r+1)
        
        return dfs(root, R)

## or use bisect to deal with this problem.

In [27]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None
class Solution:
    def getNumber(self, root: Optional[TreeNode], ops: List[List[int]]) -> int:
        re = []
        def dfs(root):
            if not root:
                return
            dfs(root.left)
            re.append(root.val)
            dfs(root.right)
        
        dfs(root)
        ret = 0
        for i, l, r in ops[::-1]:
            j = bisect.bisect_left(re, l)
            n = bisect.bisect_left(re, r)
            if j == len(re):
                continue
            if n < len(re) and re[n] == r:
                n += 1
            re = re[:j] + re[n:]
            if i == 1:
                ret += n-j
        return ret

In [26]:
a = [1,2,3,5]
bisect.bisect_left(a, 5)

3

## use segment_tree

In [29]:
class Segment_tree:
    def __init__(self, lens):
        self.len = lens
        self.Tree = [0] * (lens<<2)
        self.lazy = [0] * (lens<<2)
        
    def push_up(self, k):
        self.Tree[k] = self.Tree[k<<1] + self.Tree[(k<<1)+1]
    
    def _build(self, l, r, k):
        if l == r:
            self.Tree[k] = self.a[l]
            return  
        mid = l + (r-l)//2
        self._build(l, mid, k<<1)
        self._build(mid+1, r, (k<<1)+1)     
        self.push_up(k)
        return 
    
    def query(self, l, r, k, L, R):
        if l >= L and r <= R:
            return self.Tree[k]
        mid = l + (r-l)//2
        self.push_down(k, mid-l+1, r-mid)
        ret = 0
        if L <= mid:
            ret += self.query(l, mid, k<<1, L, R)
        if R > mid:
            ret += self.query(mid+1, r, (k<<1)+1, L, R)
        return ret
    
    def push_down(self, k, l, r):
        if self.lazy[k]:
            self.lazy[k<<1] = self.lazy[k]
            self.lazy[(k<<1)+1] = self.lazy[k]     
            self.Tree[k<<1] = l if self.lazy[k] == 1 else 0
            self.Tree[(k<<1)+1] = r if self.lazy[k] == 1 else 0
            self.lazy[k] = 0
            
    def update(self, l, r, k, L, R, C):
        if l >= L and r <= R:
            self.Tree[k] = (r-l+1) * C if C == 1 else 0
            self.lazy[k] = C
            return 
        mid = l + (r-l)//2
        self.push_down(k, mid-l+1, r-mid)
        if L <= mid:
            self.update(l, mid, k<<1, L, R, C)
        if R > mid:
            self.update(mid+1, r, (k<<1)+1, L, R, C)
        self.push_up(k)
    
    def update_interval(self, L, R, C):
        self.update(0, self.len-1, 1, L, R, C)
        
    def update_point(self, L, C):
        self.update(0, self.len-1, 1, L, L, C)        
        
    def query_interval(self, L, R):
        return self.query(0, self.len-1, 1, L, R)
    
    def query_point(self, L):
        return self.query(0, self.len-1, 1, L, L)        

In [30]:
class Solution:
    def getNumber(self, root: Optional[TreeNode], ops: List[List[int]]) -> int:
        tree_val = set()
        total_val = set()
        def dfs(root):
            if not root:
                return
            dfs(root.left)
            tree_val.add(root.val)
            total_val.add(root.val)
            dfs(root.right)
        
        dfs(root)
        for _, l, r in ops:
            total_val.add(l)
            total_val.add(r)
        total_val.sort()
        maps = {n:i for i, n in enumerate(total_val)}
        lens = len(re)
        s = Segment_tree(lens + 1)
        ret = 0
        for i, l, r in ops:
            j = maps[l]
            n = maps[r]
            if i == 1:
                s.update_interval(j, n, 1)
            else:
                s.update_interval(j, n, -1)
        for val in tree_val:
            ret += s.query_point(maps[val])
        return ret

In [31]:
class Solution:
    def getNumber(self, root: Optional[TreeNode], ops: List[List[int]]) -> int:
        re = []
        def dfs(root):
            if not root:
                return
            dfs(root.left)
            re.append(root.val)
            dfs(root.right)
        
        dfs(root)
        lens = len(re)
        s = Segment_tree(lens + 1)
        ret = 0
        for i, l, r in ops:
            j = bisect.bisect_left(re, l)
            n = bisect.bisect_left(re, r)
            if j == len(re):
                continue
            if i == 1:
                s.update_interval(j, n, 1)
            else:
                s.update_interval(j, n, -1)
        return s.query_interval(0, lens-1)

# 守护太空城

提示 1
考虑计算前 $i$ 个舱室需要的最小能量，由于题目中只有联合屏障会涉及到相邻的舱室，因此可以往 DP 上去思考。

提示 2
枚举第 $i-1$ 个舱室在哪些时间点（记作 $pre$ 集合）开启了联合屏障，这会产生如下影响（限制）：

第 $i-1$ 个舱室在不属于 $pre$ 集合的时间点，若受到陨石雨的冲击，必须开启单独屏障（因为只有单独屏障和联合屏障两种防护方式）；

第 $i$ 个舱室在属于 $pre$ 集合的时间点，由于被第 $i-1$ 个舱室的联合屏障保护，无需开启任何屏障；

第 $i$ 个舱室在不属于 $pre$ 集合的时间点，若受到陨石雨的冲击，必须选择开启单独屏障，或者选择开启联合屏障。

提示 3
将「开启联合屏障的时间点集合」当作状态中的一个维度。

In [44]:
inf = float("inf")
class Solution:
    def defendSpaceCity(self, time: List[int], position: List[int]) -> int:
        n, m = max(position), 1 << max(time)
        # 构造每个位置的时间序列
        rain = [0] * (n + 1)
        for t, p in zip(time, position):
            rain[p] |= 1 << (t - 1)
        
        union = [0] * m
        single = [0] * m
        for i in range(1, m):
            lb = i & -i
            j = i ^ lb
            lb2 = j & -j
            union[i] = union[j] + (1 if lb == (lb2 >> 1) else 3)  # lb == (lb2 >> 1) 表示两个时间点相邻
            single[i] = single[j] + (1 if lb == (lb2 >> 1) else 2)  # 递推

        f = [[inf] * m for _ in range(n + 1)]
        for i in range(n + 1):
            for j in range(m):
                pre = mask = (m - 1) ^ j
                while True:  # 枚举 j 的补集 mask 中的子集 pre
                    cost = (f[i - 1][pre] if i else 0) + union[j] + single[(mask ^ pre) & rain[i]]
                    f[i][j] = min(f[i][j], cost)
                    if pre == 0:
                        break
                    pre = (pre - 1) & mask
        return f[n][0]

https://codeforces.com/blog/entry/66715?#comment-507869

In [46]:
a = Solution().defendSpaceCity(time, position)
a

[0, 7, 3, 17]


9

 1. 枚举向右开的状态(也就是联合开的)，剩下的状态记作mask

 2.从mask枚举子集为之前i-1和i联合开的屏障状态

 3.从1,2剩下的能开屏障的状态与最小要求rain取交集，这些都单独开